In [6]:
class WTNode:
    def __init__(self,d,l,m,r):
        self.data = d
        self.left = l
        self.right = r
        self.midl = m
        self.mult = 0
    
    def __str__(self):  
        st = "("+str(self.data)+", "+str(self.mult)+") -> ["
        if self.left != None:
            st += str(self.left)
        else: st += "None"
        if self.midl != None:
            st += ", "+str(self.midl)
        else: st += ", None"
        if self.right != None:
            st += ", "+str(self.right)
        else: st += ", None"
        return st + "]"
    
class WordTree:
    def __init__(self):
        self.root = None
        self.size = 0

    def __str__(self):
        return str(self.root)
    
    def _resize(self):
        temp = [0 for i in range(2*len(self.words))]
        for i in range(len(self.words)):
            temp[i] = self.words[i]
        self.words = temp
    
    def count(self, st):
        if st == "" or st is None: 
            return None
        
        if not self.root:
            return 0
    
        return self._countRec(self.root, st)
    
    def _countRec(self, node, st):
        if len(st) == 0 or node is None: 
            return 0
        
        head = st[0]
        tail = st[1:]
        
        if head < node.data: 
            return self._countRec(node.left, st)
        elif head > node.data: 
            return self._countRec(node.right, st)
        else:
            if node.mult >= 1 and len(tail) == 0:
                return node.mult
            return self._countRec(node.midl, tail)
    
    def add(self, st):  
        if st == "" or st is None:
            return
        if self.root is None:
            self.root = WTNode(st[0], None, None, None)
        
        self.size += 1
        self._insert(self.root, st)
        return None    
    
    def _insert(self, node, st):
        if len(st) == 0:
            return node
        
        head = st[0]
        tail = st[1:]
        
        if node is None:
            node = WTNode(head, None, None, None)
        if head < node.data:
            node.left = self._insert(node.left, st)
        elif head > node.data:
            node.right = self._insert(node.right, st)
        else:
            if len(tail) == 0:
                node.mult += 1
            else:
                node.midl = self._insert(node.midl, tail)
        return node
                
    def remove(self, st):
        if st == "" or st is None or self.root is None or self.count(st) == 0: 
            return None
        
        elements = self._getPath(self.root, st)
        lastEl = elements[len(elements)-1]
        if lastEl.mult == 0:
            return 
        
        lastEl.mult -= 1
        self.size -= 1
        # more than one occurance of word still left
        if lastEl.mult > 1:
            return
                
        for i in range(len(elements)-1, -1, -1):
            curr = elements[i]
            prev = elements[i-1]
            left = curr.left
            right = curr.right
            midl = curr.midl
            
            if curr.data not in st:
                continue 
                
            if curr.mult >= 1:
                break
                
            if curr.midl is not None:
                continue
            
            # ROOT REMOVAL
            if curr is self.root:
                if curr.right:
                    self.root = curr.right
                elif curr.left:
                    self.root = curr.left
                else:
                    self.root = None
                    
            # NO CHILDREN 
            if left is None and right is None and midl is None:
                if prev.left is curr:
                    prev.left = None
                elif prev.midl is curr:
                    prev.midl = None
                elif prev.right is curr:
                    prev.right = None
            
            # ONE CHILD
            elif left is None or right is None:
                if curr is prev.left:
                    if right is None:
                        prev.left = left
                    elif left is None:
                        prev.left = right
                elif curr is prev.right:
                    if right is None:
                        prev.right = left
                    elif left is None:
                        prev.right = right
                elif curr is prev.midl:
                    if right is None:
                        prev.midl = left
                    elif left is None:
                        prev.midl = right

            # THREE CHILDREN
            else:
                gChild = right
                if curr is prev.right:
                    prev.right = gChild
                elif curr is prev.left:
                    prev.left = gChild
                else:
                    prev.midl = gChild

                if gChild is right:
                    ptr = left
                    while ptr.right is not None:
                        ptr = ptr.right

                    ptr.right = gChild.left
                    gChild.left = left
                    return  
    
    def _getPath(self, node, st):
        if len(st) == 0 or node is None: 
            return []
        
        head = st[0]
        tail = st[1:]
        
        if head < node.data: 
            return self._combine([node], self._getPath(node.left, st))
        elif head > node.data: 
            return self._combine([node], self._getPath(node.right, st))
        else:
            return self._combine([node], self._getPath(node.midl, tail))
        
    def _combine(self, A1, A2):
        res = [None for i in range(len(A1) + len(A2))]
        for i in range(len(A1)): res[i] = A1[i]
        for i in range(len(A1), len(res)): res[i] = A2[i-len(A1)]
        return res
        
    def minst(self):    
        if self.root is None: 
            return None
        
        return self._minstRec(self.root, "")
            
    def _minstRec(self, node, st):
        if node.mult > 0:
            if node.left:
                return self._minstRec(node.left, "")
            else:
                return node.data
        
        if node.left:
            return self._minstRec(node.left, "")
        elif node.midl:
            return node.data + self._minstRec(node.midl, node.data)
        elif node.right:
            return self._minstRec(node.right, "")
    
t = WordTree()
t.add("def"); t.add("car"); t.add("cat"); t.add("cat")
print(t.minst())

car
